# Lectures 7 and 8: Building Autograd engine from scratch

## Learning outcomes
1. Understanding automated differentiatiom engines at a foundational level
2. Operator overloading in OOP
3. Graph representation
4. Graph traversal

## Problem Statement
Restrictin gyourself to <font color='red'>Python's Standard Library</font>, build an <font color='red'>autograd engine</font> capable of estimating the gradients required to solve the following problem using gradient descent.

Find a point in the R<sup>2</sup> with the least average Euclidean distance to a set of arbitrary points.

**As long as my loss is scalar differentiable wrt my unknowns the methods that will be explained in this lecture are applicable to any problem**

In [2]:
from random import Random
from math import sqrt, ceil
SEED = 5

def generate_pnts(N = 1000):
    random_gen = Random(x = SEED)
    lst_x , lst_y =[],[]
    for _ in range(N):
        lst_x.append(random_gen.uniform(a = 0, b = 1))
    for _ in range(N):
        lst_y.append(random_gen.uniform(a = 0, b = 1))
    return lst_x, lst_y

# closed form analytically driven calculation of the gradient
def calc_grad(x_p, y_p, x_i, y_i):
    sum_x, sum_y = 0, 0
    n = len(x_i)
    for x, y in zip(x_i, y_i):
        inv_sqrt = ((x - x_p) ** 2 + (y - y_p) ** 2) ** (-0.5)
        sum_x += inv_sqrt * (x - x_p)
        sum_y += inv_sqrt * (y - y_p)
    return -sum_x/n, -sum_y/n

def loss(x_p, y_p, x_i, y_i):
  n = len(x_i)
  return (1 / n) * sum(
      [sqrt((x_i - x_p)**2 + (y_i - y_p)**2)
        for x_i, y_i in zip(x_i, y_i)
      ]
  )

In [3]:
data_x, data_y = generate_pnts()
x_p, y_p = 0.3, 0.3
grad_x, grad_y = calc_grad(x_p, y_p, data_x, data_y)
cur_loss = loss(x_p, y_p, data_x, data_y)
print(f"Closed form: gradient for x_p {grad_x}, gradient for y_p {grad_y} ")
print(f"Closed form: loss = {cur_loss}")

#loss for one pnt to compare to the autograd at the end (forward pass)
data_x, data_y = generate_pnts(N = 1)
x_p, y_p = 0.3, 0.3
grad_x, grad_y = calc_grad(x_p, y_p, data_x, data_y)
cur_loss = loss(x_p, y_p, data_x, data_y)
print(f"Closed form: gradient for x_p {grad_x}, gradient for y_p {grad_y} ")
print(f"Closed form: loss = {cur_loss}")


Closed form: gradient for x_p -0.3277744397151291, gradient for y_p -0.336282174741702 
Closed form: loss = 0.4430528244756474
Closed form: gradient for x_p -0.5900849147094943, gradient for y_p -0.8073411877467226 
Closed form: loss = 0.5472122517293468


## Pytorch as an example of autograd engines

In [4]:
import torch
data_pnt = torch.tensor([0.3, 0.3])
data_pnt.requires_grad = True
data_pnt.retain_grad()
data = torch.tensor([data_x, data_y])
print(data.shape)
data = data.t()
print(data.shape, data_pnt.shape)

torch.Size([2, 1])
torch.Size([1, 2]) torch.Size([2])


**Why do I need to zero the grad?**

In [5]:
for _ in range(2):
    loss_torch = torch.mean(torch.sqrt(((data - data_pnt)**2).sum(dim=1))) #foward pass
    print(f"loss calc by pytorch: {loss_torch}")
    #every .backward the gradient is accumulated aka added 
    #it is automated it doesnt know that we are repeating the row
    # it treats it as a new dtaa_pnt so it calcs gard and accumulates them
    #REMEMBER -> while calc the deriveative aka gradient wrt all points we ADDED THEM aka accumulated them that is what it is
    # doing here
    # first .backward()->gard then 2nd-> 2*grad then 3rd->3*grad and so on 
    # the lower one here is 2*grad and 3*grad
    loss_torch.backward()
    print(f"torch grad: {data_pnt.grad.data}")
    print('*' *50)
    #ZEROING
for _ in range(5):
    loss_torch = torch.mean(torch.sqrt(((data - data_pnt)**2).sum(dim=1)))
    print(f"loss calc by pytorch: {loss_torch}")
    #every .backward the gradient is accumulated aka added 
    #it is automated it doesnt know that we are repeating the row
    # it treats it as a new dtaa_pnt so it calcs gard and accumulates them
    #REMEMBER -> while calc the deriveative aka gradient wrt all points we ADDED THEM aka accumulated them that is what it is
    # doing here
    # first .backward()->grad then 2nd-> 2*grad then 3rd->3*grad and so on 
    # the lower one here is 2*grad and 3*grad
    loss_torch.backward()
    print(f"torch grad: {data_pnt.grad.data}")
    data_pnt.grad.zero_() #inplace
    print(f"torch grad after zeroing: {data_pnt.grad.data}")

loss calc by pytorch: 0.5472122430801392
torch grad: tensor([-0.5901, -0.8073])
**************************************************
loss calc by pytorch: 0.5472122430801392
torch grad: tensor([-1.1802, -1.6147])
**************************************************
loss calc by pytorch: 0.5472122430801392
torch grad: tensor([-1.7703, -2.4220])
torch grad after zeroing: tensor([0., 0.])
loss calc by pytorch: 0.5472122430801392
torch grad: tensor([-0.5901, -0.8073])
torch grad after zeroing: tensor([0., 0.])
loss calc by pytorch: 0.5472122430801392
torch grad: tensor([-0.5901, -0.8073])
torch grad after zeroing: tensor([0., 0.])
loss calc by pytorch: 0.5472122430801392
torch grad: tensor([-0.5901, -0.8073])
torch grad after zeroing: tensor([0., 0.])
loss calc by pytorch: 0.5472122430801392
torch grad: tensor([-0.5901, -0.8073])
torch grad after zeroing: tensor([0., 0.])


**What is my loss relative to my data -> forward pass**

In [25]:
#broadcasting therefore element-wise subtraction
#this is the forward pass and pytorch could calc the back grad (the next code cell)
loss_torch = torch.mean(torch.sqrt(((data - data_pnt)**2).sum(dim=1)))
#dim=0 -> across rows, dim=1 ->across columns
print(f"loss calc by pytorch: {loss_torch}")

loss calc by pytorch: 0.44305282831192017


**Notice that it is equivalent to the closed form of the loss above**

Which variable do i need to calc grad wr to? pnt therefore require and retain grad

In [26]:
loss_torch.backward()
print(f"torch grad: {data_pnt.grad.data}")

torch grad: tensor([-0.3278, -0.3363])


## Backpropagation and graph

![grads1](grads1.png)

![nodesimg](nodes2.png)

Initialization dl0/dlo = 1
reverse topological sort-> i vists every node(parent) before its children


grad2 += gard1 * local_gradient
grad1 is the parent

## Building autograd from scratch

In [54]:
#val -> value calc in forward pass
#grad initilaize to zero bcus it is the additive neutral (since I accumulate the grad)
#self-driven code -> bakteb eltest abl elcode

class comp_node:
    
    def __init__(self, val, children = [], op = "assign"):
        self.val = val
        self.children = children
        self.grad = 0
        self.op = op
        self.backward_prop = lambda : None
        
    def __to_comp_node(self, obj):
        if not isinstance(obj, comp_node):                 
            return comp_node(val = obj)
        else:
            return obj
        
    def __sub__(self, other):
        
       # if not isinstance(other, comp_node):
        #    other = comp_node(val = other)
        other = self.__to_comp_node(other)
        out =  comp_node(val = (self.val - other.val), 
                         children = [self, other], op = "subtract")
        return out
    
    #here self became the right operand
    #default was the left
    def __rsub__(self, other):
        other = self.__to_comp_node(other)
        return other - self
     #  # if not isinstance(other, comp_node):
     #   #    other = comp_node(val = other)
     #   other = self.__to_comp_node(other)
     #   out =  comp_node(val = (other.val - self.val), 
     #                    children = [self, other], op = "sub")
        
    
    def __pow__(self, exponent):
        if not isinstance(exponent, (int, float)):
            raise ValueError("Unsupported types of exponent")
        out = comp_node(val = self.val ** exponent, children = [self], op=f"power {exponent}")
        # I do not understand this line 
        #the one with the underscore is a different function
        def _backward_prop():
            self.grad += out.grad * (exponent * self.val * (exponent - 1))
        out.backward_prop = _backward_prop()
        return out
    
    #think about do i have to compare the children as well
    #ACCORDING TO THE APPLICATION
    def __eq__(self, other):
        other = self.__to_comp_node(other)
        return self.val == other.val
    
    def __add__(self, other):
        other = self.__to_comp_node(other)
        out = comp_node(val = self.val + other.val, children = [self, other], op = "add")
        return out
    
    def __radd__(self, other):
        other = self.__to_comp_node(other)
        return self +other
    
    def __mul__(self, other):
        other = self.__to_comp_node(other)
        out = comp_node(val = self.val * other.val, children = [self, other], op = "mul")
        return out
    
    def __rmul__(self, other):
        other = self.__to_comp_node(other)
        return self*other
    
    #This was the output when we printed a nde <__main__.comp_node object at 0x00000206159DA230>
    #we will overload represent aka toString in java
    # .4f -> 4 places after the decimal
    def __repr__(self):
        return f"op = {self.op} | val = {self.val:.4f} | children = {len(self.children)} | grad = {self.grad}"
    
assert comp_node(val = 5).val == 5, "Assignment failed"
assert (comp_node(val = 5) - comp_node(val = 3)).val == 2       
assert (comp_node(val = 5) - 3).val == 2
assert (3 - comp_node(val = 2)).val == 1
assert (comp_node(val = 7) - comp_node(val = 2)).val == 5

#we didnt tell comp node how to do the comparison, we must overload it
#therefore __eq__
assert (comp_node(val = 5) ** 2) == comp_node(val = 25)

#this one was here by default
assert comp_node(val = 3) + comp_node(val = 5) == comp_node(val = 8)

assert comp_node(val = 3) + 3 == 6
assert (3 + comp_node(val = 3)).val == 6

assert (comp_node(val = 3) *2).val == 6
assert comp_node(val = 3) *comp_node(val = 3) == comp_node(val = 9)

assert(comp_node(val = 3) *2).val == 6
assert(2 * comp_node(val = 3)).val == 6

**Let's begin with our forward pass for the loss function**

In [55]:
data_x, data_y = generate_pnts(N = 1)
x_p, y_p = comp_node(val = 0.3), comp_node(val = 0.3)

def loss_graph(x_p, y_p, data_x, data_y):
    I_x, I_y = x_p - data_x, y_p - data_y
    g_x, g_y = I_x**2, I_y**2
    M = g_x + g_y
    l = M ** 0.5
    #we will also return the nodes in inverse topological sort
    return l, [l, M, g_x, g_y, I_x, I_y, x_p, y_p]

#data_x and data_y are lists therefore we must use data
#_x[0] (in our simple example, we are using one point)

l, rev_topo_order = loss_graph(x_p, y_p, data_x[0], data_y[0])
print(l)

#Notice tha it is the same as the last cell before autograd engine

op = power 0.5 | val = 0.5472 | children = 1 | grad = 0


## Backward pass
* reverse topological sort of our nodes
* initialization of dlo/dlo = 1
* apply chain rule

In [57]:
data_x, data_y = generate_pnts(N = 1)
x_p, y_p = comp_node(val = 0.3), comp_node(val = 0.3)

def loss_graph(x_p, y_p, data_x, data_y):
    I_x, I_y = x_p - data_x, y_p - data_y
    g_x, g_y = I_x**2, I_y**2
    M = g_x + g_y
    l = M ** 0.5
    return l

#data_x and data_y are lists therefore we must use data
#_x[0] (in our simple example, we are using one point)

l = loss_graph(x_p, y_p, data_x[0], data_y[0])
# grad initilization
rev_topo_order[0].grad = 1
print(l)
#Notice tha it is the same as the last cell before autograd engine

for i, node in enumerate(rev_topo_order):
    print(i, node)
    

op = power 0.5 | val = 0.5472 | children = 1 | grad = 0
0 op = power 0.5 | val = 0.5472 | children = 1 | grad = 1
1 op = add | val = 0.2994 | children = 2 | grad = 0.0
2 op = power 2 | val = 0.1043 | children = 1 | grad = 0
3 op = power 2 | val = 0.1952 | children = 1 | grad = 0
4 op = subtract | val = -0.3229 | children = 2 | grad = 0.0
5 op = subtract | val = -0.4418 | children = 2 | grad = 0.0
6 op = assign | val = 0.3000 | children = 0 | grad = 0
7 op = assign | val = 0.3000 | children = 0 | grad = 0
